In [1]:
import requests
from io import BytesIO
import pandas as pd
from datetime import datetime

In [2]:
# 기본방식
today = "20220520"
page = 1

url = f"https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate={today}&page={page}"
rsep = requests.get(url)
html = BytesIO(rsep.content)

df = pd.read_html(html)[0]

df.dropna(inplace=True)
df = df.droplevel(0, axis=1)
df['날짜'] = df['날짜'].apply(lambda x: "20" +x)
df['날짜'] = df['날짜'].apply(lambda x: x.replace(".", "-"))
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)
df.head()

,개인,외국인,기관계,금융투자,보험,투신(사모),은행,기타금융기관,연기금등,기타법인
날짜,,,,,,,,,,
2022-05-20,-10412.0,1943.0,8365.0,7418.0,-3.0,210.0,-77.0,-109.0,928.0,104.0
2022-05-19,5168.0,-896.0,-4828.0,-3546.0,-20.0,-1508.0,-15.0,-66.0,326.0,556.0
2022-05-18,260.0,2973.0,-3496.0,-3179.0,-86.0,2531.0,42.0,-25.0,-2778.0,263.0
2022-05-17,-3083.0,-543.0,5409.0,4760.0,-159.0,183.0,10.0,-26.0,641.0,-1783.0
2022-05-16,650.0,9.0,-1159.0,-1048.0,-108.0,-79.0,18.0,-82.0,141.0,500.0


In [3]:
# 반복문을 사용해 크롤링 진행
# 2005년 1월 3일 까지만 사용가능
start = "20220520"
end = "20100101"
end = pd.to_datetime(end)
page = 1
while True:
    url = f"https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate={today}&sosok=&page={page}"
    rsep = requests.get(url)
    html = BytesIO(rsep.content)
    
    df = pd.read_html(html)[0]
    df.dropna(inplace=True)
    df = df.droplevel(0, axis=1)
    df['날짜'] = df['날짜'].apply(lambda x: "20" +x)
    df['날짜'] = df['날짜'].apply(lambda x: x.replace(".", "-"))
    df['날짜'] = pd.to_datetime(df['날짜'])
    df.set_index('날짜', inplace=True)
    
    if all(df.index < end):
        break
    else:
        page += 1
        
    if page == 1:
        df.to_csv('naver_finance.csv')
    else:
        df.to_csv('naver_finance.csv', header=None, mode='a')

In [4]:
# 약간의 편집
naver_finance = pd.read_csv('naver_finance.csv',index_col=0)
naver_finance.columns = df.columns
naver_finance.index = pd.to_datetime(naver_finance.index)
naver_finance.index.name='날짜'

In [5]:
naver_finance = naver_finance[naver_finance.index >= end]
naver_finance.tail()

,개인,외국인,기관계,금융투자,보험,투신(사모),은행,기타금융기관,연기금등,기타법인
날짜,,,,,,,,,,
2010-01-08,375.0,594.0,-743.0,31.0,84.0,-882.0,-375.0,123.0,277.0,-227.0
2010-01-07,-1656.0,2202.0,-855.0,312.0,-609.0,-497.0,107.0,120.0,-288.0,309.0
2010-01-06,-3470.0,3508.0,-367.0,345.0,-248.0,-553.0,81.0,-56.0,63.0,329.0
2010-01-05,-1666.0,3916.0,-2227.0,-145.0,15.0,-1469.0,105.0,2.0,-734.0,-24.0
2010-01-04,-2700.0,2387.0,-72.0,-53.0,541.0,-522.0,77.0,-6.0,-109.0,385.0


In [6]:
naver_finance.to_csv('naver_finance.csv', header=True, encoding="utf-8-sig")